# Stock Exchange Data

Daily price data for indexes tracking stock exchanges from all over the world (United States, China, Canada, Germany, Japan, and more). The data was all collected from Yahoo Finance, which had several decades of data available for most exchanges.

In this project I am going to predict the close value of daily index prices for multiple Stock Exchanges.

Dataset Link : https://www.kaggle.com/mattiuzc/stock-exchange-data

# Create Datframe

In [ ]:
!pip install opendatasets
import opendatasets as od

od.download('https://www.kaggle.com/mattiuzc/stock-exchange-data')

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: amithirpara
Your Kaggle Key: ··········


100%|██████████| 4.56M/4.56M [00:00<00:00, 32.8MB/s]

I am going to use indexData.csv file for training and validation of my models.

In [ ]:
import pandas as pd

data = pd.read_csv('/content/stock-exchange-data/indexData.csv')
data

,Index,Date,Open,High,Low,Close,Adj Close,Volume
0,NYA,1965-12-31,528.690002,528.690002,528.690002,528.690002,528.690002,0.0
1,NYA,1966-01-03,527.210022,527.210022,527.210022,527.210022,527.210022,0.0
2,NYA,1966-01-04,527.840027,527.840027,527.840027,527.840027,527.840027,0.0
3,NYA,1966-01-05,531.119995,531.119995,531.119995,531.119995,531.119995,0.0
4,NYA,1966-01-06,532.070007,532.070007,532.070007,532.070007,532.070007,0.0
...,...,...,...,...,...,...,...,...
112452,N100,2021-05-27,1241.119995,1251.910034,1241.119995,1247.069946,1247.069946,379696400.0
112453,N100,2021-05-28,1249.469971,1259.209961,1249.030029,1256.599976,1256.599976,160773400.0
112454,N100,2021-05-31,1256.079956,1258.880005,1248.140015,1248.930054,1248.930054,91173700.0
112455,N100,2021-06-01,1254.609985,1265.660034,1254.609985,1258.579956,1258.579956,155179900.0


In [ ]:
data.describe()

,Open,High,Low,Close,Adj Close,Volume
count,110253.000000,110253.000000,110253.000000,110253.000000,110253.000000,1.102530e+05
mean,7658.515222,7704.372961,7608.000422,7657.545872,7657.351729,1.273975e+09
std,9011.478913,9066.638548,8954.506981,9011.510444,9011.608900,4.315783e+09
min,54.869999,54.869999,54.869999,54.869999,54.869999,0.000000e+00
25%,1855.030029,1864.510010,1843.979980,1855.060059,1854.179565,0.000000e+00
50%,5194.089844,5226.279785,5154.049805,5194.750000,5194.750000,4.329000e+05
75%,10134.299810,10207.820310,10060.349610,10134.830080,10134.830080,1.734314e+08
max,68775.062500,69403.750000,68516.992190,68775.062500,68775.062500,9.440374e+10


In [ ]:
data.isna().sum()

Index           0
Date            0
Open         2204
High         2204
Low          2204
Close        2204
Adj Close    2204
Volume       2204
dtype: int64

Here we can see that there is some null values in dataset. This data is about stock prices, so that we can't impute the empty cells. So that I am going to drop those rows which have null values in their cells.

In [ ]:
data.dropna(subset=['Open'], inplace=True)

In [ ]:
data.isna().sum(), data.shape

(Index        0
 Date         0
 Open         0
 High         0
 Low          0
 Close        0
 Adj Close    0
 Volume       0
 dtype: int64, (110253, 8))

So after droping some rows we also have sufficient data for our models.

# Exploratory Data Analysis

In [ ]:
import plotly.express as px
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [77]:
sns.set_style('darkgrid')
matplotlib.rcParams['font.size'] = 14
matplotlib.rcParams['figure.figsize'] = (5,5)
matplotlib.rcParams['figure.facecolor'] = '#00000000'

In [ ]:
data

,Index,Date,Open,High,Low,Close,Adj Close,Volume
0,NYA,1965-12-31,528.690002,528.690002,528.690002,528.690002,528.690002,0.0
1,NYA,1966-01-03,527.210022,527.210022,527.210022,527.210022,527.210022,0.0
2,NYA,1966-01-04,527.840027,527.840027,527.840027,527.840027,527.840027,0.0
3,NYA,1966-01-05,531.119995,531.119995,531.119995,531.119995,531.119995,0.0
4,NYA,1966-01-06,532.070007,532.070007,532.070007,532.070007,532.070007,0.0
...,...,...,...,...,...,...,...,...
112452,N100,2021-05-27,1241.119995,1251.910034,1241.119995,1247.069946,1247.069946,379696400.0
112453,N100,2021-05-28,1249.469971,1259.209961,1249.030029,1256.599976,1256.599976,160773400.0
112454,N100,2021-05-31,1256.079956,1258.880005,1248.140015,1248.930054,1248.930054,91173700.0
112455,N100,2021-06-01,1254.609985,1265.660034,1254.609985,1258.579956,1258.579956,155179900.0


In [78]:
px.scatter(data,x='Open',y='Close',title='Open vs. Close')

Output hidden; open in https://colab.research.google.com to view.

From the above graph we easily see the relation between Open and Close price of idexes. There is linear relationship between Open and Close prices.

In [79]:
px.scatter(data,x='High',y='Low',title='High vs. Low')

Output hidden; open in https://colab.research.google.com to view.

Same graph as we got earlier.

In [80]:
px.scatter(data,x='Date',y='Close',title='Date vs. Close')

Output hidden; open in https://colab.research.google.com to view.

Above graph shows that as the time goes the price value of indexes is increases, but not linearly.

In [ ]:
# Execute this to save new versions of the notebook
jovian.commit(project="course-project")

[jovian] Detected Colab notebook...
[jovian] Uploading colab notebook to Jovian...
Committed successfully! https://jovian.ai/hirparaamit43/course-project


'https://jovian.ai/hirparaamit43/course-project'

# Train and Validation dataframe

Here we are working with stock market dataset, so we have to also deal with date. Because of that I am going to split train and validation dataset using date field.

In [ ]:
year = pd.to_datetime(data.Date).dt.year

train_df = data[year<2010]
val_df = data[year>=2010]

print(train_df.shape)
print(val_df.shape)

(71117, 8)
(39136, 8)


# Input and Target Columns

In [ ]:
train_df

,Index,Date,Open,High,Low,Close,Adj Close,Volume
0,NYA,1965-12-31,528.690002,528.690002,528.690002,528.690002,528.690002,0.0
1,NYA,1966-01-03,527.210022,527.210022,527.210022,527.210022,527.210022,0.0
2,NYA,1966-01-04,527.840027,527.840027,527.840027,527.840027,527.840027,0.0
3,NYA,1966-01-05,531.119995,531.119995,531.119995,531.119995,531.119995,0.0
4,NYA,1966-01-06,532.070007,532.070007,532.070007,532.070007,532.070007,0.0
...,...,...,...,...,...,...,...,...
109533,N100,2009-12-24,679.609985,680.590027,678.789978,679.340027,679.340027,40077000.0
109534,N100,2009-12-28,683.280029,685.260010,682.710022,684.559998,684.559998,0.0
109535,N100,2009-12-29,686.109985,689.830017,685.280029,687.890015,687.890015,131860500.0
109536,N100,2009-12-30,686.549988,688.530029,682.750000,683.760010,683.760010,131415800.0


In [46]:
input_cols = ['Index','Open','High','Low','Volume']
target_col = 'Close'

train_inputs = train_df[input_cols].copy()
train_targets = train_df[target_col].copy()
val_inputs = val_df[input_cols].copy()
val_targets = val_df[target_col].copy()

train_inputs.shape, train_targets.shape

((71117, 5), (71117,))

In [48]:
train_inputs.dtypes

Index      object
Open      float64
High      float64
Low       float64
Volume    float64
dtype: object

Now we are going to separate numeric and categorical columns from train_inputs.

In [50]:
import numpy as np

numeric_cols = train_inputs.select_dtypes(include=np.number).columns.tolist()
categorical_cols = train_inputs.select_dtypes('object').columns.tolist()

print(numeric_cols)
print(categorical_cols)

['Open', 'High', 'Low', 'Volume']
['Index']


In [51]:
data.corr()

,Open,High,Low,Close,Adj Close,Volume
Open,1.000000,0.999959,0.999944,0.999907,0.999907,0.138673
High,0.999959,1.000000,0.999931,0.999955,0.999955,0.138510
Low,0.999944,0.999931,1.000000,0.999960,0.999960,0.138549
Close,0.999907,0.999955,0.999960,1.000000,1.000000,0.138555
Adj Close,0.999907,0.999955,0.999960,1.000000,1.000000,0.138560
Volume,0.138673,0.138510,0.138549,0.138555,0.138560,1.000000


In [53]:
train_inputs.isna().sum()

Index     0
Open      0
High      0
Low       0
Volume    0
dtype: int64

We already drop those rows which have null values in their cells so that we don't need to impute our input dataset.

# Scale Numeric Features

In [55]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler().fit(data[numeric_cols])

train_inputs[numeric_cols] = scaler.transform(train_inputs[numeric_cols])
val_inputs[numeric_cols] = scaler.transform(val_inputs[numeric_cols]) 

In [57]:
train_inputs.describe().loc[['min','max']]

,Open,High,Low,Volume
min,0.00000,0.000000,0.00000,0.00000
max,0.56558,0.560969,0.56634,0.63531


# Encoding Categorical Data

In [58]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse=False,handle_unknown='ignore').fit(data[categorical_cols])

encoded_cols = list(encoder.get_feature_names(categorical_cols))

train_inputs[encoded_cols] = encoder.transform(train_inputs[categorical_cols])
val_inputs[encoded_cols] = encoder.transform(val_inputs[categorical_cols])

In [62]:
train_inputs

,Index,Open,High,Low,Volume,Index_000001.SS,Index_399001.SZ,Index_GDAXI,Index_GSPTSE,Index_HSI,Index_IXIC,Index_J203.JO,Index_KS11,Index_N100,Index_N225,Index_NSEI,Index_NYA,Index_SSMI,Index_TWII
0,NYA,0.006895,0.006832,0.006921,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,NYA,0.006873,0.006811,0.006899,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,NYA,0.006883,0.006820,0.006908,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,NYA,0.006930,0.006867,0.006956,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,NYA,0.006944,0.006881,0.006970,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109533,N100,0.009091,0.009023,0.009113,0.000425,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
109534,N100,0.009144,0.009090,0.009171,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
109535,N100,0.009186,0.009156,0.009208,0.001397,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
109536,N100,0.009192,0.009137,0.009171,0.001392,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


Now, let's select only numeric and encoded columns for our final input dataset.

In [64]:
X_train = train_inputs[numeric_cols+encoded_cols]
X_val = val_inputs[numeric_cols+encoded_cols]

X_train.shape, X_val.shape

((71117, 18), (39136, 18))

# Models

To predict close price we have to perform regression on this dataset. For the regraesion I am going to use 2 models:
1. Linear-Regression
2. Random-Forest-Regressor

In [87]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

li_model = LinearRegression(n_jobs=-1).fit(X_train,train_targets)
fo_model = RandomForestRegressor(n_jobs=-1,random_state=42).fit(X_train,train_targets)

In [88]:
print(li_model.score(X_train,train_targets), li_model.score(X_val,val_targets))
print(fo_model.score(X_train,train_targets), fo_model.score(X_val,val_targets))

0.9999585383504124 0.999969766694016
0.9999914547347505 0.9139713394435315


As we can see that for the linear regression train and validation acuarcy are same, but for randomforest train and validation accuracy are different.
As we saw in visualization part the realtion between two features is linear, that's why linear regression model perform best in both train and validation cases than randomforest model.

# Coefficients of Linear Regression

In [89]:
X_train.columns

Index(['Open', 'High', 'Low', 'Volume', 'Index_000001.SS', 'Index_399001.SZ',
       'Index_GDAXI', 'Index_GSPTSE', 'Index_HSI', 'Index_IXIC',
       'Index_J203.JO', 'Index_KS11', 'Index_N100', 'Index_N225', 'Index_NSEI',
       'Index_NYA', 'Index_SSMI', 'Index_TWII'],
      dtype='object')

In [90]:
li_model.coef_, li_model.intercept_

(array([-4.54009138e+04,  6.05149792e+04,  5.39342088e+04,  1.98327995e+01,
         1.64643450e+00,  2.15001958e+00,  1.58207256e-01,  1.89630619e-01,
         2.12779280e+00, -7.05160779e-02, -2.16550688e-09,  6.47773490e-02,
        -1.91052353e-01, -1.45976384e+00, -3.35225195e-01,  2.85018252e-01,
        -6.70147323e-01, -3.89517557e+00]), 55.12641035423076)

# Feature Importance of Random Forest

In [91]:
importance_df = pd.DataFrame({
    'feature' : X_train.columns,
    'importance' : fo_model.feature_importances_
}).sort_values('importance',ascending=False)

importance_df

,feature,importance
2,Low,5.084362e-01
1,High,4.915022e-01
0,Open,5.472788e-05
3,Volume,4.206719e-06
5,Index_399001.SZ,6.932409e-07
8,Index_HSI,6.110930e-07
13,Index_N225,5.570985e-07
17,Index_TWII,1.658983e-07
16,Index_SSMI,1.314933e-07
7,Index_GSPTSE,1.295131e-07


# Hyperparameter Tuning for RandomForest

### n_esimators

In [95]:
def estimators(n):
    model = RandomForestRegressor(random_state=42,n_jobs=-1,n_estimators=n).fit(X_train,train_targets)
    train_acc = model.score(X_train,train_targets)
    val_acc = model.score(X_val,val_targets)
    return {'n_estimators' : n, 'Training Accuracy' : train_acc, 'Validation Accuracy' : val_acc}

esti_df = pd.DataFrame([estimators(n) for n in range(40,80,10)])
esti_df

,n_estimators,Training Accuracy,Validation Accuracy
0,40,0.999991,0.914010
1,50,0.999991,0.914023
2,60,0.999991,0.914017
3,70,0.999991,0.914014


From the above values of n_estimators we can say that we got same accuarcy.

### max_depth and max_leaf_node

In [99]:
def dl(md,lf):
  model = RandomForestRegressor(n_jobs=-1,random_state=42,max_depth=md,max_leaf_nodes=lf).fit(X_train,train_targets)
  train_acc = model.score(X_train,train_targets)
  val_acc = model.score(X_val,val_targets)
  return {'max_depth':md,'max_leaf_nodes':lf,'Training Accuracy':train_acc,'Validation Accuracy':val_acc}

best_df = pd.DataFrame([dl(md,lf) for md in range(1,21) for lf in range(100,200,10)])
best_df

,max_depth,max_leaf_nodes,Training Accuracy,Validation Accuracy
0,1,100,0.673678,0.332791
1,1,110,0.673678,0.332791
2,1,120,0.673678,0.332791
3,1,130,0.673678,0.332791
4,1,140,0.673678,0.332791
...,...,...,...,...
195,20,150,0.999917,0.909649
196,20,160,0.999919,0.909878
197,20,170,0.999921,0.910047
198,20,180,0.999923,0.910370


Now let's try some random values of max_depth and max_leaf node

In [116]:
model = RandomForestRegressor(n_jobs=-1,random_state=42,n_estimators=1000,max_depth=2000,max_leaf_nodes=2000).fit(X_train,train_targets)
acc = model.score(X_val,val_targets)
acc

0.9137532267229339

# Final Model Performance

In [117]:
li_score = li_model.score(X_val,val_targets) # for Linear Regression
fo_score = model.score(X_val,val_targets) # for random Forest Regression
print("Final Linear Regression model performance is : ",li_score)
print("Final Random Forest Regression model performance is : ",fo_score)

Final Linear Regression model performance is :  0.999969766694016
Final Random Forest Regression model performance is :  0.9137532267229339


# Sample Predictions

In [128]:
# For Linear Regression
list(li_model.predict(X_val)[:10]), val_targets[:10]

([7312.098133744642,
  7344.183393395817,
  7374.914405671056,
  7354.296261935906,
  7401.320576415574,
  7460.3194069833335,
  7363.5612157123605,
  7415.17453751585,
  7444.504109638009,
  7352.111506066652],
 11077    7326.740234
 11078    7354.870117
 11079    7377.700195
 11080    7393.930176
 11081    7425.350098
 11082    7449.049805
 11083    7370.450195
 11084    7430.140137
 11085    7448.520020
 11086    7356.790039
 Name: Close, dtype: float64)

In [129]:
# For Random Forest Regression
list(model.predict(X_val)[:10]), val_targets[:10]

([7282.585795372046,
  7341.42546374815,
  7367.447889917807,
  7358.2099181820595,
  7393.695463923299,
  7442.4573318233415,
  7392.674277451679,
  7413.870550185133,
  7437.672911232381,
  7375.789267539062],
 11077    7326.740234
 11078    7354.870117
 11079    7377.700195
 11080    7393.930176
 11081    7425.350098
 11082    7449.049805
 11083    7370.450195
 11084    7430.140137
 11085    7448.520020
 11086    7356.790039
 Name: Close, dtype: float64)